In [1]:
import csv
import pickle
from datetime import date, datetime, timedelta
import numpy as np
import math

## Convert row ftse file data to pickle

In [2]:
# ToDo: move all methods to a separate file and import them

def convert_txt_to_dict(filename):
    content = []

    with open(filename, newline='') as f:
        reader = csv.reader(f)
        for row in reader:
            content.append(row)

    data_ftse_100_dict = {}
    tickers_list = [ticker[:-2] for ticker in content[0][1:]]
    data_ftse_100_dict = {}

    for row in content[1:]:
        data_ftse_100_dict[row[0]] = {}
        for i, ticker in enumerate(tickers_list):
            data_ftse_100_dict[row[0]][ticker] = row[i + 1]
    return data_ftse_100_dict

    
txt_filename = 'ftse_100_time_data.csv'
path_to_save = "./data_ftse_100_dict_04032017.p"

data_ftse_100_dict = convert_txt_to_dict(txt_filename)

# pickle.dump(data_ftse_100_dict, open(path_to_save, "wb"))

## Library to interpolate ftse dataset

How to write `add_dates` method
http://stackoverflow.com/questions/3240458/how-to-increment-the-day-in-datetime-python

In [3]:
def datetime_to_date(datetime_date):
    datetime_date.date().year
    str_date = '{}-{}-{}'.format(datetime_date.date().strftime('%Y'), datetime_date.date().strftime('%m'), datetime_date.date().strftime('%d'))
    return str_date
    
def date_to_datetime(date):
    return datetime.strptime(date, '%Y-%m-%d')

def find_start_end_dates(ticker, dates_dict):
    # this method looks when a company started to be in ftse 100. Assume that some dates may not have dictionaries
    # with ticker. No ticker in keys.
    start = ""
    end = ""
    for date, stock_values in dates_dict.items():
        if ticker in stock_values.keys():
            if stock_values[ticker]:
                # check if start is empty
                if not start:
                    start = date
                else:
                    end = date
    
    return start, end # dates in datetime format

def get_dates_range(start_date, end_date):
    # assume start_date and end_date are in datetime format
    dates_range = []
    delta = end_date - start_date
    date_index = start_date
    for i in range(delta.days + 1):
        dates_range.append(date_index)
        date_index += timedelta(days=1)
    return dates_range

def add_interpolated_values(dates_range, ticker, dates_dict):
    modified_dates_dict = dates_dict
    start_indicator = dates_range[0]
    end_indicator = dates_range[-1]
    interpolated_value = (float(modified_dates_dict[datetime_to_date(start_indicator)][ticker]) + float(modified_dates_dict[datetime_to_date(end_indicator)][ticker])) / 2
    for date in dates_range[1:-2]:
        # Initialise new date
        if datetime_to_date(date) not in list(modified_dates_dict.keys()):
            modified_dates_dict[datetime_to_date(date)] = {}
        modified_dates_dict[datetime_to_date(date)][ticker] = interpolated_value

    return modified_dates_dict # add average values using start_indicator and end_indicator
    
def add_missing_date(missing_date, complete_dates_dict, ticker):
    #print("here")
    #print(len(list(complete_dates_dict.keys())))
    if missing_date not in list(complete_dates_dict.keys()):
        complete_dates_dict[missing_date] = {}
        complete_dates_dict[missing_date][ticker] = ""
    else:
        if ticker not in list(complete_dates_dict[missing_date].keys()):
            complete_dates_dict[missing_date][ticker] = ""
    return complete_dates_dict

def interpolate(dates_dict, ticker, start_date, end_date):
    # assume start_date and end_date are in datetime format
    dates_range = get_dates_range(start_date, end_date)
    start_indicator = dates_range[0]
    end_indicator = dates_range[0]
    dates_dict_interp = dates_dict
    for date in dates_range[1:]:
        dates_dict_interp = add_missing_date(datetime_to_date(date), dates_dict_interp, ticker)
        #print("start", datetime_to_date(start_indicator))
        #print("end", datetime_to_date(end_indicator))
        #print("date", datetime_to_date(date))
        #print("ticker_value", dates_dict_interp[datetime_to_date(date)][ticker])
        if not dates_dict_interp[datetime_to_date(date)][ticker]:
            end_indicator = date
            #print(dates_dict_interp[datetime_to_date(date)][ticker])
            #print(datetime_to_date(date))
        else:
            if (date - start_indicator).days > 1:
                #print("start", dates_dict_interp[datetime_to_date(start_indicator)][ticker])
                #print("end", dates_dict_interp[datetime_to_date(date)][ticker])
                dates_range_interp = get_dates_range(start_indicator, date)
                dates_dict_interp = add_interpolated_values(dates_range_interp, ticker, dates_dict_interp)
            else:
                start_indicator = date
                end_indicator = date
    return dates_dict_interp

def get_existing_companies(date_point, data_ftse_100):
    tickers_list = list(data_ftse_100['2012-07-11'].keys())
    valid_tickers = []
    for ticker in tickers_list:
        start, end = find_start_end_dates(ticker, data_ftse_100)
        if date_to_datetime(date_point) > date_to_datetime(start):
            valid_tickers.append(ticker)
    return valid_tickers

def interpolate_all(tickers_lst, data_ftse, date_init, date_end):
    data_ftse_100_dict_interp = data_ftse
    for ticker in tickers_lst:
        print(ticker)
        data_ftse_100_dict_interp = interpolate(data_ftse_100_dict_interp, ticker, date_to_datetime(date_init), date_to_datetime(date_end))
    
    return data_ftse_100_dict_interp

In [4]:
start, end = find_start_end_dates("III", data_ftse_100_dict)
print(start, end)

1993-03-31 2015-04-29


In [5]:
valid_tickers = get_existing_companies("2001-04-29", data_ftse_100_dict)

print(len(valid_tickers))
print(valid_tickers)

79
['AAL', 'ABF', 'ADN', 'AGK', 'AHT', 'ANTO', 'ARM', 'AV', 'AZN', 'BA', 'BAB', 'BARC', 'BATS', 'BDEV', 'BG', 'BLND', 'BLT', 'BNZL', 'BP', 'BT-A', 'CCL', 'CNA', 'CPG', 'CPI', 'CRH', 'DGE', 'EZJ', 'GKN', 'GSK', 'HMSO', 'HSBA', 'III', 'IMT', 'INTU', 'ITV', 'JMAT', 'KGF', 'LAND', 'LGEN', 'LLOY', 'MGGT', 'MKS', 'MRW', 'NG', 'NXT', 'OML', 'PRU', 'PSN', 'PSON', 'RB', 'RBS', 'RDSB', 'REL', 'RIO', 'RR', 'RRS', 'RSA', 'SAB', 'SBRY', 'SDR', 'SGE', 'SHP', 'SKY', 'SMIN', 'SN', 'SSE', 'STAN', 'STJ', 'SVT', 'TPK', 'TSCO', 'TW', 'ULVR', 'UU', 'VOD', 'WEIR', 'WOS', 'WPP', 'WTB']


In [6]:

data_ftse_interp = interpolate_all(valid_tickers[5:7], data_ftse_100_dict, '2009-07-13', '2010-07-11')
    

ANTO
ARM


In [7]:
dat_range = get_dates_range(date_to_datetime('2012-07-11'), date_to_datetime('2013-07-11'))

for dat in dat_range:
    if datetime_to_date(dat) in list(data_ftse_100_dict.keys()):
        print(data_ftse_100_dict[datetime_to_date(dat)]["ANTO"], "ANTO", data_ftse_100_dict[datetime_to_date(dat)]["ARM"], "ARM")
    if datetime_to_date(dat) in list(data_ftse_100_dict.keys()):
        print("interpolated", data_ftse_100_dict[datetime_to_date(dat)]["ANTO"], "ANTO", data_ftse_interp[datetime_to_date(dat)]["ARM"], datetime_to_date(dat))

1077.0 ANTO 475.9 ARM
interpolated 1077.0 ANTO 475.9 2012-07-11
1075.0 ANTO 486.9 ARM
interpolated 1075.0 ANTO 486.9 2012-07-12
1058.0 ANTO 490.0 ARM
interpolated 1058.0 ANTO 490.0 2012-07-13
1074.0 ANTO 492.1 ARM
interpolated 1074.0 ANTO 492.1 2012-07-16
1076.0 ANTO 488.0 ARM
interpolated 1076.0 ANTO 488.0 2012-07-17
1057.0 ANTO 465.9 ARM
interpolated 1057.0 ANTO 465.9 2012-07-18
1076.0 ANTO 485.0 ARM
interpolated 1076.0 ANTO 485.0 2012-07-19
1086.0 ANTO 494.0 ARM
interpolated 1086.0 ANTO 494.0 2012-07-20
1048.0 ANTO 496.1 ARM
interpolated 1048.0 ANTO 496.1 2012-07-23
1030.0 ANTO 484.4 ARM
interpolated 1030.0 ANTO 484.4 2012-07-24
1039.0 ANTO 504.5 ARM
interpolated 1039.0 ANTO 504.5 2012-07-25
1063.0 ANTO 530.0 ARM
interpolated 1063.0 ANTO 530.0 2012-07-26
1071.0 ANTO 540.0 ARM
interpolated 1071.0 ANTO 540.0 2012-07-27
1076.0 ANTO 554.0 ARM
interpolated 1076.0 ANTO 554.0 2012-07-30
1089.0 ANTO 553.0 ARM
interpolated 1089.0 ANTO 553.0 2012-07-31
1083.0 ANTO 554.5 ARM
interpolated 1083.

## Calculate correlations

In [10]:
def get_components_distance(time_window, time_start, time_end, ticker_1, ticker_2, dates_dict):
    components_distance = {}
    dates_range = get_dates_range(date_to_datetime(time_start), date_to_datetime(time_end))
    
    for date in dates_range:
        s_vector_1 = get_s_vector(time_window, ticker_1, dates_dict, date)
        s_vector_2 = get_s_vector(time_window, ticker_2, dates_dict, date)
    return np.corrcoef([s_vector_1, s_vector_2]) # dict where each key is time and value is distance

def get_s_vector(time_window, ticker, dates_dict, initial_time):
    s_vector = []
    end_time = initial_time + timedelta(days=time_window)
    dates_window = get_dates_range(initial_time, end_time)
    for date in dates_window:
        s_vector.append(get_s(date, ticker, dates_dict))
    return s_vector

def get_s(time, ticker, dates_dict):
    s = np.log(float(dates_dict[datetime_to_date(time + timedelta(days=1))][ticker])) - np.log(float(dates_dict[datetime_to_date(time)][ticker]))
    return s

ticker_1 = valid_tickers[5:][0]
ticker_2 = valid_tickers[5:][1]  
#ticker_1 = "ARM"
#ticker_2 = "ANTO" 
#print(np.log(float(data_ftse_100_dict_interp[datetime_to_date(date_to_datetime('2013-04-01') + timedelta(days=1))][ticker])))
dist = get_components_distance(200, '2009-07-13', '2009-07-15', ticker_1, ticker_2, data_ftse_interp)
print(dist)
#numpy.corrcoef

[0.016122538450521695, 0.052471051737453145, 0.029124451279143315, 0.020730108946075632, 0.15425922799561409, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 